In [11]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
from google.colab import files

uploaded = files.upload()


Saving market_external_db.portfolio_market_news.json to market_external_db.portfolio_market_news.json


In [19]:
import shutil
import os

uploaded_filename = list(uploaded.keys())[0]

drive_path = f"/content/drive/MyDrive/{uploaded_filename}"

shutil.move(uploaded_filename, drive_path)

print("Saved to:", drive_path)


Saved to: /content/drive/MyDrive/market_external_db.portfolio_market_news.json


In [ ]:
import json

file_path = "/content/drive/MyDrive/market_news_2026_01_21.json"

data = []
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            data.append(json.loads(line))

print("Articles loaded:", len(data))


Articles loaded: 2900


In [ ]:
array_path = "/content/drive/MyDrive/market_news_2026_01_21_array.json"

with open(array_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("Saved array JSON to:", array_path)


Saved array JSON to: /content/drive/MyDrive/market_news_2026_01_21_array.json


In [ ]:
data[0].keys()


dict_keys(['_id', 'link', 'ai_org', 'ai_region', 'ai_summary', 'ai_tag', 'article_id', 'category', 'content', 'country', 'created_at', 'creator', 'datatype', 'description', 'duplicate', 'fetched_at', 'image_url', 'keywords', 'language', 'pubDate', 'pubDateTZ', 'sentiment', 'sentiment_stats', 'source_icon', 'source_id', 'source_name', 'source_priority', 'source_url', 'title', 'video_url'])

In [ ]:
import json
from pprint import pprint

pprint(data[0])


{'_id': {'$oid': '69708433473a7cc58efd8243'},
 'ai_org': 'ONLY AVAILABLE IN CORPORATE PLANS',
 'ai_region': 'ONLY AVAILABLE IN CORPORATE PLANS',
 'ai_summary': 'Timothy Busfield was released from jail in New Mexico after '
               'Judge David A. Murphy ordered his release while awaiting trial '
               'in a high-profile sex abuse case. The decision allowed '
               'Busfield to walk out without posting bail, leading to an '
               'emotional moment as his wife, Melissa Gilbert, broke down in '
               'tears. The case has drawn significant Hollywood attention.',
 'ai_tag': 'ONLY AVAILABLE IN PROFESSIONAL AND CORPORATE PLANS',
 'article_id': '355d6b8e1abb905398292cf3493b0191',
 'category': ['top', 'crime'],
 'content': 'Timothy Busfield is out of jail after a New Mexico judge ordered '
            'his release, sparking a dramatic courtroom moment as his wife, '
            'Melissa Gilbert, broke down in tears. Judge David A. Murphy ruled '
      

In [ ]:
texts = [
    f"{d.get('title','')}. {d.get('description','')}"
    for d in data
]

print("Sample text:\n", texts[0][:300])


Sample text:
 Timothy Busfield Walks Free; Melissa Gilbert Breaks Down In Tears. Timothy Busfield is out of jail after a New Mexico judge ordered his release, sparking a dramatic courtroom moment as his wife, Melissa Gilbert, broke down in tears. Judge David A. Murphy ruled the Emmy-winning actor and director wou


In [ ]:
!pip install -q sentence-transformers faiss-cpu scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 64.9 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = model.encode(
    texts,
    show_progress_bar=True,
    convert_to_numpy=True
)

print("Embedding shape:", embeddings.shape)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/91 [00:00<?, ?it/s]

Embedding shape: (2900, 384)


In [ ]:
import faiss

faiss.normalize_L2(embeddings)


In [ ]:
dim = embeddings.shape[1]

index = faiss.IndexFlatIP(dim)  # cosine similarity via inner product
index.add(embeddings)

print("Vectors indexed:", index.ntotal)


Vectors indexed: 2900


In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(
    eps=0.35,          # similarity threshold
    min_samples=3,
    metric="cosine"
)

labels = dbscan.fit_predict(embeddings)


In [ ]:
from collections import Counter

cluster_counts = Counter(labels)
cluster_counts


Counter({np.int64(-1): 1619,
         np.int64(0): 8,
         np.int64(1): 17,
         np.int64(2): 8,
         np.int64(3): 123,
         np.int64(4): 3,
         np.int64(5): 3,
         np.int64(62): 19,
         np.int64(6): 82,
         np.int64(17): 4,
         np.int64(7): 12,
         np.int64(61): 3,
         np.int64(8): 3,
         np.int64(9): 3,
         np.int64(10): 3,
         np.int64(11): 3,
         np.int64(12): 11,
         np.int64(13): 121,
         np.int64(14): 3,
         np.int64(15): 37,
         np.int64(16): 6,
         np.int64(18): 5,
         np.int64(19): 14,
         np.int64(20): 10,
         np.int64(21): 8,
         np.int64(22): 4,
         np.int64(37): 6,
         np.int64(23): 6,
         np.int64(24): 7,
         np.int64(25): 28,
         np.int64(26): 6,
         np.int64(27): 6,
         np.int64(28): 4,
         np.int64(29): 3,
         np.int64(30): 6,
         np.int64(31): 5,
         np.int64(32): 3,
         np.int64(33): 20,
     

In [ ]:
clusters = {}

for label, doc in zip(labels, data):
    clusters.setdefault(label, []).append(doc["title"])

for label, titles in clusters.items():
    if label == -1:
        continue
    print(f"\nCluster {label} | size={len(titles)}")
    for t in titles[:5]:
        print(" •", t)



Cluster 0 | size=8
 • Noida Techie Death: ‘They Told Me Not to Speak’, Witness Alleges Police Pressure
 • Noida Techie’s Death: How Two Viral Videos by Key Witness Told Two Different Stories
 • Noida techie death news: From car's fatal fall in waterlogged trench, SIT probe to realtor's arrest | Top 10 updates
 • Noida techie death: Developer accused of negligence arrested
 • Water-Filled Pit Left Unsecured For Over Three Years Before Noida Techie’s Death: Report

Cluster 1 | size=17
 • Israel’s Netanyahu accepts invitation to join Trump-led 'Board of Peace'
 • Israel’s Netanyahu agrees to join Trump’s Board of Peace
 • Israel’s Benjamin Netanyahu Accepts Trump’s Board Of Peace Invitation Amid Gaza Governance Talks
 • Invitation accepted: Netanyahu agrees to join Trump’s Board of Peace
 • Israel's Netanyahu agrees to join Trump's Board of Peace

Cluster 2 | size=8
 • India-EU trade deal faces new challenge from carbon lobby, non-tariff barriers: Jefferies
 • Spain backs stronger India 

# Tagging part

In [ ]:
!pip install -q spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 100.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [30]:
# ================================
# NIFTY 200 COMPANY MAPPING
# ================================

NIFTY_200 = [
    # NIFTY 50
    "Adani Enterprises","Adani Ports","Apollo Hospitals","Asian Paints","Axis Bank",
    "Bajaj Auto","Bajaj Finance","Bajaj Finserv","BEL","Bharti Airtel","Cipla",
    "Coal India","Dr Reddy's Laboratories","Eicher Motors","Eternal","Grasim",
    "HCL Technologies","HDFC Bank","HDFC Life","Hindalco","HUL","ICICI Bank",
    "IndiGo","Infosys","ITC","Jio Financial","JSW Steel","Kotak Mahindra Bank",
    "L&T","Mahindra & Mahindra","Maruti Suzuki","Max Healthcare","NTPC",
    "Nestle India","ONGC","Power Grid","Reliance Industries","SBI Life","SBI",
    "Shriram Finance","Sun Pharma","Tata Consultancy Services","Tata Consumer Products",
    "Tata Motors","Tata Steel","Tech Mahindra","Titan","Trent",
    "UltraTech Cement","Wipro",

    # NIFTY NEXT 50 + OTHERS
    "Adani Energy Solutions","Adani Green Energy","Adani Power","Ambuja Cements",
    "Bank of Baroda","Bharat Petroleum","Britannia","Canara Bank","CG Power",
    "Cholamandalam","Divi's Laboratories","DLF","DMart","GAIL","Godrej Consumer Products",
    "HAL","Havells","Hindustan Zinc","Hyundai","ICICI Lombard","Indian Hotels",
    "Indian Oil","Info Edge","IRFC","Jindal Steel","JSW Energy","LIC","Lodha",
    "LTIMindtree","Mazagon Dock Shipbuilders","Pidilite","Power Finance",
    "PNB","REC","Samvardhana Motherson","Shree Cement","Tata Power",
    "Torrent Pharma","TVS Motors","United Spirits","Varun Beverages","Vedanta",
    "Zydus Lifesciences",

    # EXTENDED
    "360 One WAM","ACC","Alkem Laboratories","Ashok Leyland","Astral",
    "AU Small Finance Bank","Aurobindo Pharma","Bank of India","Bharat Dynamics",
    "Bharat Forge","BHEL","Biocon","Blue Star","Cochin Shipyard","Coforge",
    "Container Corporation","Coromandel","Dabur","Dixon","Exide","Federal Bank",
    "Fortis Healthcare","Glenmark","GMR Airports","Godfrey Phillips","Godrej Properties",
    "Hero MotoCorp","Hindustan Petroleum","HUDCO","IDFC First Bank","Indian Bank",
    "Indraprastha Gas","Indus Towers","IndusInd Bank","IRB Infrastructure","IRCTC",
    "IREDA","ITC Hotels","Jubilant FoodWorks","Kalyan Jewellers","KPIT",
    "LIC Housing Finance","Lupin","M&M Finance","Mankind Pharma","Marico",
    "Max Financial Services","Motilal Oswal","Mphasis","MRF","Muthoot Finance",
    "NALCO","NHPC","NMDC","Nykaa","Oil India","Oracle Financial Services",
    "Page Industries","Patanjali Foods","Paytm","Persistent Systems","Policybazaar",
    "Polycab","Prestige Estates","RVNL","SAIL","SBI Card","SRF",
    "Supreme Industries","Suzlon","Swiggy","Tata Communications","Tata Technologies",
    "Torrent Power","Tube Investments","Union Bank","UPL","Vodafone Idea",
    "Voltas","Yes Bank"
]
COMPANY_LIST = NIFTY_200


In [31]:
ALL_COMPANIES = sorted(set(COMPANY_LIST ))


In [ ]:
!pip install rapidfuzz


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 53.1 MB/s eta 0:00:00


In [32]:
# ================================
# REFINED & EXPANDED SECTOR KEYWORDS
# ================================

SECTOR_KEYWORDS = {
    "Stock Market": [
        "stock", "stocks", "share", "shares", "equity", "equities",
        "sensex", "nifty", "bse", "nse",
        "ipo", "listing", "listed",
        "market rally", "market fall", "market crash",
        "valuation", "price target", "brokerage", "rating upgrade",
        "q1 results", "q2 results", "q3 results", "q4 results"
    ],

    "Banking & Finance": [
        "bank", "banks", "banking",
        "loan", "loans", "credit", "deposit",
        "interest rate", "repo rate", "reverse repo",
        "nbfc", "microfinance", "lender",
        "rbi", "central bank",
        "npa", "bad loan", "capital adequacy"
    ],

    "Insurance": [
        "insurance", "insurer",
        "policy", "premium", "claim",
        "life insurance", "health insurance",
        "general insurance", "sum assured"
    ],

    "Fintech": [
        "fintech", "digital payments",
        "upi", "wallet", "payment gateway",
        "bnpl", "neobank", "digital lending"
    ],

    "IT": [
        "it services", "software services",
        "cloud computing", "data center",
        "artificial intelligence", "machine learning",
        "saas", "enterprise software",
        "cybersecurity", "digital transformation"
    ],

    "Telecom": [
        "telecom", "telecommunications",
        "5g", "4g", "spectrum",
        "mobile network", "broadband",
        "subscriber base", "arpu"
    ],

    "Energy": [
        "energy sector", "power sector",
        "thermal power", "hydropower",
        "energy demand", "energy capacity"
    ],

    "Oil & Gas": [
        "oil", "gas", "crude oil",
        "petroleum", "refinery",
        "upstream", "downstream",
        "lng", "natural gas"
    ],

    "Renewable Energy": [
        "renewable energy",
        "solar power", "wind power",
        "green energy", "clean energy",
        "solar capacity", "wind capacity"
    ],

    "Power & Utilities": [
        "electricity", "power grid",
        "transmission", "distribution",
        "discom", "utility services"
    ],

    "Automobile": [
        "automobile", "automotive",
        "car", "cars", "vehicle", "vehicles",
        "two wheeler", "four wheeler",
        "auto sales", "vehicle sales"
    ],

    "EV & Batteries": [
        "electric vehicle", "ev",
        "battery", "lithium ion",
        "charging station", "ev policy"
    ],

    "Aviation": [
        "airline", "airlines",
        "aviation", "airport",
        "flight operations", "air traffic"
    ],

    "Infrastructure": [
        "infrastructure", "infra project",
        "construction", "road project",
        "highway", "bridge", "metro rail"
    ],

    "Real Estate": [
        "real estate", "property market",
        "housing project", "residential project",
        "commercial property", "developer"
    ],

    "Metals & Mining": [
        "metal", "metals",
        "steel", "aluminium", "copper",
        "mining", "ore", "coal production"
    ],

    "Cement": [
        "cement", "clinker",
        "cement capacity", "cement demand"
    ],

    "FMCG": [
        "fmcg", "consumer goods",
        "daily essentials", "packaged goods",
        "brand portfolio", "volume growth"
    ],

    "Retail & E-commerce": [
        "retail", "retailer",
        "ecommerce", "online retail",
        "store expansion", "consumer demand"
    ],

    "Healthcare": [
        "healthcare", "hospital",
        "medical services", "clinical care",
        "health infra"
    ],

    "Pharmaceuticals": [
        "pharma", "pharmaceutical",
        "drug", "medicine",
        "clinical trial", "usfda approval"
    ],

    "Chemicals & Fertilizers": [
        "chemical", "chemicals",
        "fertilizer", "fertilisers",
        "specialty chemicals"
    ],

    "Agri & Food": [
        "agriculture", "agri",
        "farm sector", "crop",
        "msp", "minimum support price",
        "food processing"
    ],

    "Logistics & Shipping": [
        "logistics", "supply chain",
        "warehouse", "freight",
        "shipping", "cargo"
    ],

    "Defence & Aerospace": [
        "defence", "defense",
        "military", "armed forces",
        "aerospace", "weapons system"
    ],

    "Media & Entertainment": [
        "media", "entertainment",
        "film industry", "movie",
        "ott platform", "streaming"
    ],

    "Education": [
        "education", "school",
        "college", "university",
        "exam", "admission"
    ],

    "Macro Economy": [
        "economy", "economic growth",
        "gdp", "inflation",
        "fiscal deficit", "budget",
        "trade deficit", "current account"
    ],

    "Commodities": [
        "gold", "silver",
        "commodity prices",
        "precious metals"
    ]
}


In [33]:
# ================================
# BUSINESS KEYWORDS (200+)
# ================================

BUSINESS_KEYWORDS = {
    # --- Stock Market & Trading ---
    "stock", "stocks", "share", "shares", "market", "markets", "equity", "equities",
    "ipo", "listing", "listed", "delisting", "bullish", "bearish",
    "nifty", "sensex", "dow", "nasdaq", "s&p",
    "52-week high", "52-week low", "volume", "turnover",
    "price target", "rating", "upgrade", "downgrade",
    "brokerage", "broker", "analyst", "research report",
    "technical chart", "support", "resistance",

    # --- Financial Performance ---
    "revenue", "income", "profit", "loss", "net profit", "gross profit",
    "earnings", "earnings growth", "ebitda", "margin", "operating margin",
    "quarter", "quarterly", "annual", "fy", "fy24", "fy25",
    "q1", "q2", "q3", "q4",
    "results", "financial results", "guidance", "outlook",
    "valuation", "market cap", "capitalisation",

    # --- Banking & Finance ---
    "bank", "banks", "banking", "nbfc", "lender", "lending",
    "loan", "loans", "credit", "deposit", "interest", "interest rate",
    "repo rate", "reverse repo", "monetary policy",
    "rbi", "central bank",
    "insurance", "policy", "premium", "claim",
    "mutual fund", "mutual funds", "sip", "aum",
    "investment", "investments", "investor", "institutional investor",
    "private equity", "venture capital", "vc funding",

    # --- Corporate Actions ---
    "merger", "mergers", "acquisition", "acquisitions",
    "stake", "stake sale", "buyout", "takeover",
    "joint venture", "strategic partnership",
    "divestment", "demerger", "spin-off",
    "funding", "fund raise", "capital raise",
    "rights issue", "preferential issue",

    # --- Economy & Policy (Business Context) ---
    "economy", "economic", "gdp", "growth", "inflation",
    "fiscal", "budget", "union budget",
    "tax", "gst", "duty", "customs",
    "subsidy", "incentive", "pli scheme",
    "trade", "exports", "imports",
    "foreign investment", "fdi", "fpi",

    # --- Infrastructure ---
    "infrastructure", "infra", "construction",
    "road", "highway", "expressway",
    "bridge", "tunnel", "metro", "rail",
    "airport", "ports", "shipping", "logistics",
    "warehouse", "supply chain",
    "industrial corridor",

    # --- Energy & Utilities ---
    "power", "electricity", "energy",
    "oil", "gas", "crude", "lng",
    "renewable", "renewables",
    "solar", "wind", "green energy",
    "coal", "thermal", "hydro",
    "transmission", "distribution",
    "power plant", "refinery",

    # --- Manufacturing & Core Sectors ---
    "manufacturing", "production", "capacity",
    "factory", "plant", "unit",
    "steel", "aluminium", "metal", "metals",
    "cement", "clinker",
    "mining", "ore", "coal mine",
    "engineering", "epc",

    # --- Technology ---
    "technology", "tech", "it", "it services",
    "software", "platform", "product",
    "cloud", "saas", "paas", "iaas",
    "ai", "artificial intelligence", "ml", "machine learning",
    "data", "analytics", "cybersecurity",
    "semiconductor", "chip", "fab",
    "digital transformation",

    # --- Telecom ---
    "telecom", "telecom services",
    "5g", "4g", "spectrum",
    "network", "broadband",
    "mobile services",

    # --- Pharma & Healthcare ---
    "pharma", "pharmaceutical", "drug", "medicine",
    "clinical trial", "clinical",
    "healthcare", "hospital", "diagnostics",
    "medical devices", "biotech",

    # --- FMCG, Retail, Consumer ---
    "fmcg", "consumer goods",
    "retail", "retailer",
    "store", "outlet",
    "brand", "brands",
    "demand", "consumption",
    "pricing", "price hike",

    # --- Auto & Mobility ---
    "automobile", "auto", "vehicle", "vehicles",
    "car", "two-wheeler", "four-wheeler",
    "ev", "electric vehicle",
    "battery", "charging",
    "mobility",

    # --- Real Estate ---
    "real estate", "property",
    "housing", "residential",
    "commercial property",
    "office space",
    "developer", "project launch",

    # --- Startups & New Age ---
    "startup", "startups",
    "unicorn", "soonicorn",
    "founder", "co-founder",
    "burn rate", "unit economics",
    "scaling", "expansion",

    # --- Defence & Aerospace (Business side) ---
    "defence manufacturing",
    "aerospace",
    "weapons system",
    "defence deal",
    "indigenous production",

    # --- Agriculture (Business angle) ---
    "agriculture", "agri business",
    "fertilizer", "fertilisers",
    "crop procurement",
    "msp", "minimum support price",
    "food processing",

    # --- Employment & Corporate ---
    "hiring", "recruitment",
    "jobs", "layoffs",
    "workforce", "employees",
    "salary", "wages",
    "esop", "stock options"
}

# ================================
# NON-BUSINESS HINTS
# ================================

NON_BUSINESS_HINTS = {
    "murder", "rape", "killed", "death", "suicide",
    "accident", "crash", "fire", "violence",
    "politics", "political", "election", "minister",
    "chief minister", "prime minister", "parliament",
    "court", "supreme court", "high court", "judge",
    "police", "arrest", "crime", "fir",
    "actor", "actress", "film", "movie", "cinema",
    "celebrity", "bollywood", "hollywood",
    "cricket", "football", "match", "tournament",
    "ipl", "fifa", "wwe", "ufc",
    "religion", "temple", "mosque", "church",
    "protest", "rally", "riot"
}





In [34]:
# ================================
# ADDITIONAL STRONG BUSINESS KEYWORDS (APPEND)
# ================================

BUSINESS_KEYWORDS |= {
    # --- Financial Statements & Metrics ---
    "profit after tax", "pat margin", "net margin",
    "topline", "bottomline",
    "year-on-year", "yoy", "quarter-on-quarter", "qoq",
    "return on equity", "roe",
    "return on capital employed", "roce",
    "free cash flow", "cash flow",
    "debt-to-equity", "leverage",
    "cost optimization", "cost pressure",

    # --- Capital Markets & Trading (Advanced) ---
    "block deal", "bulk deal",
    "open interest", "oi",
    "futures", "options", "derivatives",
    "call option", "put option",
    "short selling", "long position",
    "volatility index", "india vix",
    "market breadth",
    "upper circuit", "lower circuit",

    # --- Corporate Governance & Filings ---
    "board meeting", "board approval",
    "exchange disclosure",
    "sebi approval", "regulatory filing",
    "annual general meeting", "agm",
    "extraordinary general meeting", "egm",
    "shareholder approval",
    "insider trading",

    # --- Banking (Advanced) ---
    "gross npa", "net npa",
    "slippage", "credit cost",
    "capital adequacy", "crar",
    "tier 1 capital", "tier 2 capital",
    "provision coverage ratio", "pcr",
    "loan disbursement",
    "deposit growth",

    # --- IPO / Fundraising ---
    "anchor investors",
    "offer for sale", "ofs",
    "price band",
    "subscription status",
    "grey market premium", "gmp",
    "listing gains",

    # --- M&A / Restructuring ---
    "scheme of arrangement",
    "asset monetisation",
    "strategic divestment",
    "minority stake",
    "controlling stake",
    "hostile takeover",

    # --- Macro Economy (Hard Business) ---
    "industrial production",
    "iip data",
    "manufacturing pmi",
    "services pmi",
    "current account deficit",
    "cad",
    "trade deficit",
    "forex reserves",
    "balance of payments",

    # --- Government Business Schemes ---
    "capital expenditure",
    "public sector undertaking",
    "psu",
    "disinvestment",
    "strategic sale",
    "government tender",
    "bid process",

    # --- Technology (Enterprise / Business) ---
    "data centre",
    "cloud migration",
    "enterprise software",
    "erp implementation",
    "crm platform",
    "digital payments",
    "fintech platform",

    # --- Energy / Infra (Business Language) ---
    "capacity addition",
    "installed capacity",
    "power purchase agreement",
    "ppa",
    "tariff order",
    "fuel supply agreement",

    # --- FMCG / Retail (Business Metrics) ---
    "same store sales",
    "sssg",
    "volume growth",
    "price-led growth",
    "rural demand",
    "urban consumption",

    # --- Employment / Corporate Actions ---
    "headcount reduction",
    "workforce rationalisation",
    "employee cost",
    "attrition rate",
    "hiring freeze",

    # --- Startups / VC (Advanced) ---
    "series a", "series b", "series c",
    "pre-series a",
    "down round", "up round",
    "term sheet",
    "cap table",
    "exit valuation",
}

# ================================
# ADDITIONAL NON-BUSINESS BLOCKERS (APPEND)
# ================================

NON_BUSINESS_HINTS |= {
    # --- Politics / Governance ---
    "assembly election",
    "lok sabha", "rajya sabha",
    "poll campaign",
    "vote share",
    "political rally",
    "party workers",

    # --- Sports (Expanded) ---
    "test match", "odi series",
    "t20 match",
    "goalkeeper", "batsman", "bowler",
    "runs", "wickets",
    "points table",
    "semi final", "final match",

    # --- Crime / Law ---
    "charge sheet",
    "custodial death",
    "police custody",
    "court hearing",
    "legal battle",
    "bail plea",

    # --- Weather / Climate ---
    "cyclonic storm",
    "heavy rainfall",
    "heatwave",
    "cold wave",
    "weather forecast",

    # --- Culture / Celebrities ---
    "music album",
    "film release",
    "box office",
    "trailer launch",
    "fashion show",
    "award ceremony",

    # --- Health / Personal ---
    "hospitalised",
    "medical condition",
    "health bulletin",
    "recovery update",
}



# ================================
# ADDITIONAL STRONG BUSINESS KEYWORDS (APPEND)
# ================================

BUSINESS_KEYWORDS |= {
    # --- Financial Statements & Metrics ---
    "profit after tax", "pat margin", "net margin",
    "topline", "bottomline",
    "year-on-year", "yoy", "quarter-on-quarter", "qoq",
    "return on equity", "roe",
    "return on capital employed", "roce",
    "free cash flow", "cash flow",
    "debt-to-equity", "leverage",
    "cost optimization", "cost pressure",

    # --- Capital Markets & Trading (Advanced) ---
    "block deal", "bulk deal",
    "open interest", "oi",
    "futures", "options", "derivatives",
    "call option", "put option",
    "short selling", "long position",
    "volatility index", "india vix",
    "market breadth",
    "upper circuit", "lower circuit",

    # --- Corporate Governance & Filings ---
    "board meeting", "board approval",
    "exchange disclosure",
    "sebi approval", "regulatory filing",
    "annual general meeting", "agm",
    "extraordinary general meeting", "egm",
    "shareholder approval",
    "insider trading",

    # --- Banking (Advanced) ---
    "gross npa", "net npa",
    "slippage", "credit cost",
    "capital adequacy", "crar",
    "tier 1 capital", "tier 2 capital",
    "provision coverage ratio", "pcr",
    "loan disbursement",
    "deposit growth",

    # --- IPO / Fundraising ---
    "anchor investors",
    "offer for sale", "ofs",
    "price band",
    "subscription status",
    "grey market premium", "gmp",
    "listing gains",

    # --- M&A / Restructuring ---
    "scheme of arrangement",
    "asset monetisation",
    "strategic divestment",
    "minority stake",
    "controlling stake",
    "hostile takeover",

    # --- Macro Economy (Hard Business) ---
    "industrial production",
    "iip data",
    "manufacturing pmi",
    "services pmi",
    "current account deficit",
    "cad",
    "trade deficit",
    "forex reserves",
    "balance of payments",

    # --- Government Business Schemes ---
    "capital expenditure",
    "public sector undertaking",
    "psu",
    "disinvestment",
    "strategic sale",
    "government tender",
    "bid process",

    # --- Technology (Enterprise / Business) ---
    "data centre",
    "cloud migration",
    "enterprise software",
    "erp implementation",
    "crm platform",
    "digital payments",
    "fintech platform",

    # --- Energy / Infra (Business Language) ---
    "capacity addition",
    "installed capacity",
    "power purchase agreement",
    "ppa",
    "tariff order",
    "fuel supply agreement",

    # --- FMCG / Retail (Business Metrics) ---
    "same store sales",
    "sssg",
    "volume growth",
    "price-led growth",
    "rural demand",
    "urban consumption",

    # --- Employment / Corporate Actions ---
    "headcount reduction",
    "workforce rationalisation",
    "employee cost",
    "attrition rate",
    "hiring freeze",

    # --- Startups / VC (Advanced) ---
    "series a", "series b", "series c",
    "pre-series a",
    "down round", "up round",
    "term sheet",
    "cap table",
    "exit valuation",
}

# ================================
# ADDITIONAL NON-BUSINESS BLOCKERS (APPEND)
# ================================

NON_BUSINESS_HINTS |= {
    # --- Politics / Governance ---
    "assembly election",
    "lok sabha", "rajya sabha",
    "poll campaign",
    "vote share",
    "political rally",
    "party workers",

    # --- Sports (Expanded) ---
    "test match", "odi series",
    "t20 match",
    "goalkeeper", "batsman", "bowler",
    "runs", "wickets",
    "points table",
    "semi final", "final match",

    # --- Crime / Law ---
    "charge sheet",
    "custodial death",
    "police custody",
    "court hearing",
    "legal battle",
    "bail plea",

    # --- Weather / Climate ---
    "cyclonic storm",
    "heavy rainfall",
    "heatwave",
    "cold wave",
    "weather forecast",

    # --- Culture / Celebrities ---
    "music album",
    "film release",
    "box office",
    "trailer launch",
    "fashion show",
    "award ceremony",

    # --- Health / Personal ---
    "hospitalised",
    "medical condition",
    "health bulletin",
    "recovery update",
}
# ================================
# ADDITIONAL STRONG BUSINESS KEYWORDS (APPEND)
# ================================

BUSINESS_KEYWORDS |= {
    # --- Financial Statements & Metrics ---
    "profit after tax", "pat margin", "net margin",
    "topline", "bottomline",
    "year-on-year", "yoy", "quarter-on-quarter", "qoq",
    "return on equity", "roe",
    "return on capital employed", "roce",
    "free cash flow", "cash flow",
    "debt-to-equity", "leverage",
    "cost optimization", "cost pressure",

    # --- Capital Markets & Trading (Advanced) ---
    "block deal", "bulk deal",
    "open interest", "oi",
    "futures", "options", "derivatives",
    "call option", "put option",
    "short selling", "long position",
    "volatility index", "india vix",
    "market breadth",
    "upper circuit", "lower circuit",

    # --- Corporate Governance & Filings ---
    "board meeting", "board approval",
    "exchange disclosure",
    "sebi approval", "regulatory filing",
    "annual general meeting", "agm",
    "extraordinary general meeting", "egm",
    "shareholder approval",
    "insider trading",

    # --- Banking (Advanced) ---
    "gross npa", "net npa",
    "slippage", "credit cost",
    "capital adequacy", "crar",
    "tier 1 capital", "tier 2 capital",
    "provision coverage ratio", "pcr",
    "loan disbursement",
    "deposit growth",

    # --- IPO / Fundraising ---
    "anchor investors",
    "offer for sale", "ofs",
    "price band",
    "subscription status",
    "grey market premium", "gmp",
    "listing gains",

    # --- M&A / Restructuring ---
    "scheme of arrangement",
    "asset monetisation",
    "strategic divestment",
    "minority stake",
    "controlling stake",
    "hostile takeover",

    # --- Macro Economy (Hard Business) ---
    "industrial production",
    "iip data",
    "manufacturing pmi",
    "services pmi",
    "current account deficit",
    "cad",
    "trade deficit",
    "forex reserves",
    "balance of payments",

    # --- Government Business Schemes ---
    "capital expenditure",
    "public sector undertaking",
    "psu",
    "disinvestment",
    "strategic sale",
    "government tender",
    "bid process",

    # --- Technology (Enterprise / Business) ---
    "data centre",
    "cloud migration",
    "enterprise software",
    "erp implementation",
    "crm platform",
    "digital payments",
    "fintech platform",

    # --- Energy / Infra (Business Language) ---
    "capacity addition",
    "installed capacity",
    "power purchase agreement",
    "ppa",
    "tariff order",
    "fuel supply agreement",

    # --- FMCG / Retail (Business Metrics) ---
    "same store sales",
    "sssg",
    "volume growth",
    "price-led growth",
    "rural demand",
    "urban consumption",

    # --- Employment / Corporate Actions ---
    "headcount reduction",
    "workforce rationalisation",
    "employee cost",
    "attrition rate",
    "hiring freeze",

    # --- Startups / VC (Advanced) ---
    "series a", "series b", "series c",
    "pre-series a",
    "down round", "up round",
    "term sheet",
    "cap table",
    "exit valuation",
}

# ================================
# ADDITIONAL NON-BUSINESS BLOCKERS (APPEND)
# ================================

NON_BUSINESS_HINTS |= {
    # --- Politics / Governance ---
    "assembly election",
    "lok sabha", "rajya sabha",
    "poll campaign",
    "vote share",
    "political rally",
    "party workers",

    # --- Sports (Expanded) ---
    "test match", "odi series",
    "t20 match",
    "goalkeeper", "batsman", "bowler",
    "runs", "wickets",
    "points table",
    "semi final", "final match",

    # --- Crime / Law ---
    "charge sheet",
    "custodial death",
    "police custody",
    "court hearing",
    "legal battle",
    "bail plea",

    # --- Weather / Climate ---
    "cyclonic storm",
    "heavy rainfall",
    "heatwave",
    "cold wave",
    "weather forecast",

    # --- Culture / Celebrities ---
    "music album",
    "film release",
    "box office",
    "trailer launch",
    "fashion show",
    "award ceremony",

    # --- Health / Personal ---
    "hospitalised",
    "medical condition",
    "health bulletin",
    "recovery update",
}



# ================================
# STRONG NON-BUSINESS INTENT BLOCKERS (APPEND)
# ================================

NON_BUSINESS_HINTS |= {
    # --- Accidents / Death ---
    "road accident", "died on the spot", "succumbed to injuries",
    "fatal accident", "hit by a vehicle",

    # --- Law & Order ---
    "police said", "according to police",
    "arrested on charges", "remanded to custody",
    "investigation is underway",

    # --- Politics (Hard Block) ---
    "press conference said",
    "slammed the opposition",
    "addressing party workers",
    "political roadmap",
    "election campaign",

    # --- Sports (Hard Block) ---
    "won the match",
    "lost the match",
    "scored runs",
    "took wickets",
    "pole position",
    "injured reserve",
    "season opener",

    # --- Weather / Environment ---
    "weather department",
    "imd said",
    "fog conditions",
    "aqi levels",
    "pollution levels",

    # --- Culture / Lifestyle ---
    "went viral on social media",
    "instagram post",
    "celebrated birthday",
    "wedding ceremony",
    "award night",

    # --- Judiciary / Courts ---
    "supreme court said",
    "high court observed",
    "court hearing",
    "bail was granted",
}




BUSINESS_KEYWORDS |= {
    # --- Business Strategy / Expansion ---
    "business strategy",
    "growth strategy",
    "expansion plans",
    "capacity expansion",
    "market outlook",
    "industry outlook",
    "long term outlook",

    # --- Product & Innovation ---
    "product launch",
    "new product",
    "product lineup",
    "innovation strategy",
    "technology roadmap",

    # --- Hiring / Workforce (Business context) ---
    "hiring drive",
    "campus hiring",
    "fresher hiring",
    "salary hike",
    "compensation revision",

    # --- Infrastructure & Projects ---
    "project commissioned",
    "commercial operations",
    "electricity production",
    "power generation",
    "solar project",
    "wind project",

    # --- Market & Industry Analysis ---
    "market size",
    "market value",
    "industry growth",
    "sector growth",
    "demand outlook",

    # --- Enterprise & AI ---
    "enterprise adoption",
    "ai strategy",
    "enterprise ai",
    "digital strategy",

    # --- Retail / Consumer Strategy ---
    "brand strategy",
    "product positioning",
    "consumer demand",
}


In [35]:
# ================================
# BUSINESS RELEVANCE FUNCTION
# ================================

def is_business_relevant(text: str) -> bool:
    if not text:
        return False

    t = text.lower()

    business_hits = sum(1 for k in BUSINESS_KEYWORDS if k in t)
    non_business_hits = sum(1 for k in NON_BUSINESS_HINTS if k in t)

    # Strong business signal
    # Strong business signal AND low non-business intent
    if business_hits >= 2 and non_business_hits == 0:
      return True


    # Single business signal with limited noise
    if business_hits == 1 and non_business_hits <= 3:
        return True

    return False


In [36]:
# Sector classification
import spacy
nlp = spacy.load("en_core_web_sm")

def detect_sector(text):
    if not text:
        return "Non-Business"

    text_l = text.lower()
    scores = {sector: 0 for sector in SECTOR_KEYWORDS}

    # Keyword scoring
    for sector, keys in SECTOR_KEYWORDS.items():
        for k in keys:
            if k in text_l:
                scores[sector] += 2   # strong signal

    # spaCy lemma soft scoring
    doc = nlp(text)
    for token in doc:
        lemma = token.lemma_.lower()
        for sector, keys in SECTOR_KEYWORDS.items():
            if lemma in keys:
                scores[sector] += 1   # weaker signal

    # Pick best sector
    best_sector = max(scores, key=scores.get)
    best_score = scores[best_sector]

    # Threshold to avoid noise
    if best_score >= 2:
        return best_sector

    return "Non-Business"



In [37]:
# ================================
# COMPANY DETECTION
# ================================

import re
from difflib import SequenceMatcher

# ================================
# CONFIG
# ================================

FUZZY_THRESHOLD = 0.7
MAX_COMPANIES = 5

# ================================
# GUARDS
# ================================

GENERIC_COMPANY_BLACKLIST = {
    "Target", "Block", "Progressive", "Gap", "ACC", "Nationwide"
}

BUSINESS_CONTEXT_WORDS = {
    "share", "shares", "stock", "stocks", "revenue", "profit", "loss",
    "ipo", "results", "quarter", "q1", "q2", "q3", "q4",
    "market", "markets", "investor", "investment", "valuation",
    "deal", "acquisition", "merger", "funding", "order", "contract",
    "bank", "loan", "credit", "earnings", "financial"
}

# ================================
# HELPERS
# ================================

def similarity(a, b):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

def sentence_split(text):
    return re.split(r'[.!?\n]', text)

def exact_word_match(company, text):
    pattern = r'\b' + re.escape(company.lower()) + r'\b'
    return re.search(pattern, text.lower()) is not None

def has_business_context(text: str, company: str) -> bool:
    t = text.lower()
    c = company.lower()

    idx = t.find(c)
    if idx == -1:
        return False

    window = t[max(0, idx - 200): idx + 200]
    return any(w in window for w in BUSINESS_CONTEXT_WORDS)

# ================================
# PRECOMPUTE COMPANY TOKENS (RUN ONCE)
# ================================

COMPANY_TOKENS = {
    c: set(re.findall(r'\b[a-z]+\b', c.lower()))
    for c in COMPANY_LIST
}

# ================================
# FAST & BALANCED COMPANY DETECTOR
# ================================

def detect_companies_fast(text, company_list):
    if not text:
        return []

    text_l = text.lower()
    text_tokens = set(re.findall(r'\b[a-z]+\b', text_l))
    sentences = sentence_split(text)

    detected = []

    for company in company_list:

        # Skip dangerous generic names
        if company in GENERIC_COMPANY_BLACKLIST:
            continue

        tokens = COMPANY_TOKENS.get(company)
        if not tokens:
            continue

        # Stage 1: token gate (FAST)
        if not (tokens & text_tokens):
            continue

        # Stage 2: exact match (STRONG)
        if exact_word_match(company, text):
          if has_business_context(text, company):
            detected.append((company, 1.0))
          continue


        # Stage 3: fuzzy match (LIMITED + SAFE)
        for s in sentences:
            if not is_business_relevant(s):
                continue

            if similarity(company.lower(), s.lower()) >= FUZZY_THRESHOLD:
                if has_business_context(text, company):
                    detected.append((company, 0.7))
                break

    detected.sort(key=lambda x: x[1], reverse=True)
    return [c for c, _ in detected[:MAX_COMPANIES]]


In [58]:
print(similarity("Infosys","Infosys Limited"))

0.6363636363636364


In [52]:
def tag_article(title, description, content):
    text = f"{title or ''} {description or ''} {content or ''}".strip()

    if not text:
        return {"company": [], "sector": "Non-Business"}

    if "only available in paid plans" in text.lower():
        return {"company": [], "sector": "Non-Business"}

    # Detect companies (signal only)
    detected_companies = detect_companies_fast(text, ALL_COMPANIES)

    # Decide business intent
    is_business = False

    if is_business_relevant(text):
        is_business = True
    elif detected_companies:
        for s in sentence_split(text):
            if is_business_relevant(s):
                is_business = True
                break

    if not is_business:
        return {"company": [], "sector": "Non-Business"}

    # Only NOW keep companies
    companies = detected_companies

    sector = detect_sector(text)

    return {
        "company": companies,
        "sector": sector
    }


In [39]:
FILE_PATH = "/content/drive/MyDrive/market_external_db.portfolio_market_news.json"

with open(FILE_PATH, "r", encoding="utf-8") as f:
    for i in range(5):
        print(f.readline())


[{

  "_id": {

    "$oid": "696f4084473a7cc58efd6cc3"

  },

  "link": "https://www.edexlive.com/news/ai-can-help-radiologists-improve-precision-delivering-high-quality-care-to-people-mohit-soni-executive-director-of-soni-group-of-hospitals",



In [40]:
import json

FILE_PATH = "/content/drive/MyDrive/market_external_db.portfolio_market_news.json"

articles = []
bad_lines = 0

with open(FILE_PATH, "r", encoding="utf-8") as f:
    for idx, line in enumerate(f, 1):
        line = line.strip()
        if not line:
            continue
        try:
            articles.append(json.loads(line))
        except json.JSONDecodeError:
            bad_lines += 1
            # Uncomment below to debug specific lines
            # print(f"Bad JSON at line {idx}: {line[:100]}")

print("Articles loaded:", len(articles))
print(" Bad lines skipped:", bad_lines)


Articles loaded: 93847
 Bad lines skipped: 1068176


In [41]:
import json

FILE_PATH = "/content/drive/MyDrive/market_external_db.portfolio_market_news.json"

with open(FILE_PATH, "r", encoding="utf-8") as f:
    articles = json.load(f)

print("Total articles loaded:", len(articles))


Total articles loaded: 25414


In [42]:
import re

COMPANY_TOKENS = {
    company: set(re.findall(r'\b[a-z]+\b', company.lower()))
    for company in COMPANY_LIST
}


In [43]:
import random

sample = random.sample(articles, min(50, len(articles)))

print("\n=== RUNNING RANDOM 20 ARTICLES TEST ===\n")

for i, a in enumerate(sample, 1):
    text_blob = f"{a.get('title','')} {a.get('description','')} {a.get('content','')}".lower()

    if "only available in paid plans" in text_blob:
        continue

    result = tag_article(
        a.get("title"),
        a.get("description"),
        a.get("content")
    )

    print("=" * 80)
    print(f"ARTICLE {i}")
    print("-" * 80)

    preview = (a.get("content") or a.get("description") or a.get("title") or "")
    print("CONTENT :")
    print(preview[:300])

    print("\nCOMPANY:", result["company"])
    print("SECTOR:", result["sector"])

print("\n=== TEST COMPLETED ===")



=== RUNNING RANDOM 20 ARTICLES TEST ===

ARTICLE 1
--------------------------------------------------------------------------------
CONTENT :
Hyderabad, Jan 20 (PTI) Telangana BRS MLA T Harish Rao was on Tuesday questioned by the SIT in the case related to alleged phone tapping during the previous BRS regime. Harish Rao appeared before the SIT officials at the Jubilee Hills police station here at about 11.30 am and was questioned till 6 p

COMPANY: []
SECTOR: Non-Business
ARTICLE 2
--------------------------------------------------------------------------------
CONTENT :
Domestic equity benchmarks Sensex and Nifty began Wednesday’s session on a negative note after Tuesday’s sharp decline, the steepest single-day percentage fall since May 2025. Market sentiment remained fragile amid persistent global trade-related uncertainties and continued selling pressure from for

COMPANY: ['BEL', 'Bharti Airtel', 'ICICI Bank', 'L&T', 'Trent']
SECTOR: Stock Market
ARTICLE 3
------------------------

# Clustering part


In [44]:
import json
import random

path = "/content/drive/MyDrive/market_external_db.portfolio_market_news.json"
articles = json.load(open(path))

# Randomly sample 10k articles
SAMPLE_SIZE = 10_000
articles_10k = random.sample(articles, min(SAMPLE_SIZE, len(articles)))

texts = [
    f"{a.get('title','')} {a.get('description','')} {a.get('content','')}"
    for a in articles_10k
]

print("Total texts:", len(texts))


Total texts: 10000


In [45]:
import torch
from sentence_transformers import SentenceTransformer

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model = SentenceTransformer(
    "all-MiniLM-L6-v2",
    device=device
)

embeddings = model.encode(
    texts,
    batch_size=128,              # good for Colab GPU
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

print("Embedding shape:", embeddings.shape)


Using device: cuda


Batches:   0%|          | 0/79 [00:00<?, ?it/s]

Embedding shape: (10000, 384)


In [46]:
import torch
from sentence_transformers import SentenceTransformer

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model = SentenceTransformer(
    "all-MiniLM-L6-v2",
    device=device
)

embeddings = model.encode(
    texts,
    batch_size=128,              # good for Colab GPU
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

print("Embedding shape:", embeddings.shape)


Using device: cuda


Batches:   0%|          | 0/79 [00:00<?, ?it/s]

Embedding shape: (10000, 384)


In [53]:
enriched_articles = []

for a in articles:
    result = tag_article(
        a.get("title"),
        a.get("description"),
        a.get("content")
    )

    enriched_articles.append({
        "text": f"{a.get('title','')} {a.get('description','')} {a.get('content','')}".strip(),
        "company": result["company"],
        "sector": result["sector"]
    })

print("Total articles:", len(enriched_articles))
print("Business articles:", sum(1 for a in enriched_articles if a["sector"] != "Non-Business"))


Total articles: 25414
Business articles: 6048


In [54]:
business_articles = [
    a for a in enriched_articles
    if a["sector"] != "Non-Business"
]

texts = [a["text"] for a in business_articles]

print("Articles going to clustering:", len(texts))


Articles going to clustering: 6048


In [55]:
from sklearn.cluster import DBSCAN
from collections import defaultdict

# embeddings MUST align with `texts`
# (you already created embeddings for these texts)

dbscan = DBSCAN(
    eps=0.35,
    min_samples=5,
    metric="cosine"
)

labels = dbscan.fit_predict(embeddings)

clusters = defaultdict(list)

for label, article in zip(labels, business_articles):
    if label != -1:
        clusters[label].append(article)

print("Total clusters:", len(clusters))

# 🔍 Print 5 articles from first 5 clusters
for cid, items in list(clusters.items())[:5]:
    print(f"\n🟦 CLUSTER {cid} | Sector: {items[0]['sector']} | Companies: {items[0]['company']}")
    for a in items[:5]:
        print("-", a["text"][:120])


Total clusters: 128

🟦 CLUSTER 0 | Sector: Healthcare | Companies: ['Infosys']
- AI can enhance precision and care quality in radiology: Mohit Soni "AI can help radiologists improve precision, deliveri
- December quarter earnings muted so far as labour code impact weighs on results So far, around 10 companies from the Nift
- WEF 2026: MP govt woos Peace Invest for sustainable energy projects Madhya Pradesh government met with Peace Invest at t
- Bank Nifty extends decline for 2nd day; PSU bank index falls 1.2%: What should investors do? Axis Securities said that B
- Union Budget 2026: BEL, CESC, M&M Fin, Amara Raja among 15 stocks to watch Axis Securities sees Ultratech Cement, Maruti

🟦 CLUSTER 1 | Sector: Banking & Finance | Companies: ['Axis Bank', 'Infosys', 'Reliance Industries', 'Wipro']
- World News | MP Govt Interacts with Geneva-based Peace Invest in Davos for Water, Energy Investments Get latest articles
- TRAI examining Navi Mumbai airport RoW issue: Chairman TRAI Chairman A

In [56]:
from collections import defaultdict

clusters = defaultdict(list)

for label, article in zip(labels, business_articles):
    if label != -1:   # ignore noise
        clusters[label].append(article)

print("Total clusters:", len(clusters))

for cid in sorted(clusters):
    items = clusters[cid]
    print("\n" + "="*90)
    print(f"🟦 CLUSTER {cid}")
    print(f"Sector      :", items[0]['sector'])
    print(f"Companies   :", items[0]['company'])
    print(f"Size        :", len(items))
    print("-"*90)

    for a in items[:5]:   # 🔒 limit to 5 articles
        print("•", a["text"][:150])


Total clusters: 128

🟦 CLUSTER 0
Sector      : Healthcare
Companies   : ['Infosys']
Size        : 1994
------------------------------------------------------------------------------------------
• AI can enhance precision and care quality in radiology: Mohit Soni "AI can help radiologists improve precision, delivering high-quality care to people
• December quarter earnings muted so far as labour code impact weighs on results So far, around 10 companies from the Nifty 50 index have reported their
• WEF 2026: MP govt woos Peace Invest for sustainable energy projects Madhya Pradesh government met with Peace Invest at the Davos WEF 2026 to discuss s
• Bank Nifty extends decline for 2nd day; PSU bank index falls 1.2%: What should investors do? Axis Securities said that Bank Nifty is centered around t
• Union Budget 2026: BEL, CESC, M&M Fin, Amara Raja among 15 stocks to watch Axis Securities sees Ultratech Cement, Maruti Suzuki India Ltd, Bharti Airt

🟦 CLUSTER 1
Sector      : Banking & Fina